In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

#Library
# ---
# For scrape
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Options driver
from selenium.webdriver.chrome.options import Options

# For dataframes
import pandas as pd
from pandas import json_normalize

# For simulate human behavior.
import time
from time import sleep
import random

# Clear data
import unidecode
#unaccented_string = unidecode.unidecode(accented_string)

# Manipulate Json files
import json
import re
import numpy as np
from time import strptime

In [49]:
# Setting
options= Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# we open our page
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [C:\Users\Anzony\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [50]:
# Dropdown button list
dropdown_button_year = driver.find_element_by_id( "ctl00_cphContent_rAnio")
dropdown_button_year.click()

# List of years options
list_year = driver.find_element_by_id( "ctl00_cphContent_rAnio_DropDown" )
years_list = list_year.find_elements_by_tag_name('li')
list_year_options = len(years_list)

# Dropdown button list
dropdown_button_month = driver.find_element_by_id( "ctl00_cphContent_rMes")
dropdown_button_month.click()

# # List of months options
list_month = driver.find_element_by_id( "ctl00_cphContent_rMes_DropDown" )
months_list = list_month.find_elements_by_tag_name('li')
list_month_options = len(months_list)
driver.find_element_by_id("ctl00_cphContent_btnConsultaMensual").click()
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

In [51]:
year_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_year_options).tolist() )
month_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_month_options).tolist() )

In [52]:
years_info_list = []
months_info_list = []
tables_info_list = []

for year_number in year_numbers_opt:
    
    # Dropdown button list
    dropdown_button_year = driver.find_element_by_id( "ctl00_cphContent_rAnio")
    dropdown_button_year.click()

    # List of years options
    time.sleep(0.5)
    driver.find_element_by_xpath( f"/html/body/form/div[1]/div/ul/li[{year_number + 1 }]" ).click()
    year_text = driver.find_element_by_id( "ctl00_cphContent_rAnio").text
        
    for month_number in month_numbers_opt:
        
        # Dropdown button list
        dropdown_button_month = driver.find_element_by_id( "ctl00_cphContent_rMes")
        dropdown_button_month.click()

        # # List of months options
        time.sleep(0.5)
        driver.find_element_by_xpath( f"/html/body/form/div[1]/div/ul/li[{month_number + 1 }]" ).click()
        month_text = driver.find_element_by_id( "ctl00_cphContent_rMes").text
        print( year_text )
        print( "\n",month_text )
        
        # Click consultar
        driver.find_element_by_id("ctl00_cphContent_btnConsultaMensual").click()
        time.sleep(5)
        
        year_float = int(year_text)
        
        try:
            # Switch elements
            driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element_by_xpath( "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # Cleaning tablr
            table_clean_1 = table_pd[1].iloc[5:, :].copy().reset_index( drop = True )

            # Changing switch
            driver.switch_to.default_content()

            print("Table Tipo 1 - Cambiar de Frame y una tabla simple")

        except:
            print("No table Tipo 1")
            table_clean_1 = "Null"
            
            
        try:
            # Switch elements
            driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
            section_tab_table = driver.find_element_by_xpath( "/html/body/form/table/tbody" )
            table_html = section_tab_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # generate table objects
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_2 = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_2.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 2 - Cambiar de Frame y una tabla Compleja")
            
             # Changing switch
            driver.switch_to.default_content()
            
        except:
            print("No table Tipo 2")
            table_clean_2 = "Null"
            

        try:
            # Get tables
            section_id_table = driver.find_element_by_id( "ctl00_cphContent_rpgActualMn_OT" )
            table_html = section_id_table.get_attribute('outerHTML')
            table_pd = pd.read_html( table_html )

            # generate table objects
            table0, table1 = table_pd[0], table_pd[1]
            table_clean_3 = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean_3.insert( loc = 0, column= col_name, value = col_values)
            
            print("Table Tipo 3 - No Cambiar de Frame y una tabla Compleja")
            
        except:
            print("No table Tipo 3")
            table_clean_3 = "Null"
            
            
        # Check what to save
        if ( ( isinstance( table_clean_3, str ) == False ) and ( table_clean_3.shape[1] > 5 ) ):
            
            # Data with no frame
            table_clean_3.to_excel( fr"B_RawData/sbs/B_RawData/cajas_municipales/table_clean_{year_text}_{month_text}.xlsx", index = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 3" )
            print("guardo3")
            
        elif ( ( isinstance( table_clean_2, str ) == False ) and ( table_clean_2.shape[1] > 5 ) ):
            
            # Data with frame and 2 tables
            table_clean_2.to_excel( fr"B_RawData/sbs/B_RawData\cajas_municipales/table_clean_{year_text}_{month_text}.xlsx", index = False, header = False )
            # To break
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 2" )
            print("guardo2")
            
        elif ( ( isinstance( table_clean_1, str ) == False ) and ( table_clean_1.shape[1] > 5 ) ):
            # Data with frame and 1 table
            table_clean_1.to_excel( fr"B_RawData/sbs/B_RawData\cajas_municipales/table_clean_{year_text}_{month_text}.xlsx", index = False , header = False )
            
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "Table 1" )
            print("guardo1")
            
        # No table information
        # Go to a previous day
        else:
            # Day ago
            years_info_list.append( year_float)
            months_info_list.append( month_text )
            tables_info_list.append( "No Table" )
            
        driver.switch_to.default_content()

2021

 Febrero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Enero
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Marzo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Abril
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Mayo
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Junio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Julio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Agosto
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Setiembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2021

 Octubr

No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Junio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Julio
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Agosto
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Setiembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Octubre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Noviembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2016

 Diciembre
No table Tipo 1
No table Tipo 2
Table Tipo 3 - No Cambiar de Frame y una tabla Compleja
guardo3
2015

 Febrero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2015

 Enero
Table T

No table Tipo 2
No table Tipo 3
guardo1
2010

 Noviembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2010

 Diciembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Febrero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Enero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Marzo
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Abril
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Mayo
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Junio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Julio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2009

 Agosto
T

2003

 Mayo
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Junio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Julio
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Agosto
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Setiembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Octubre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Noviembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2003

 Diciembre
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
guardo1
2002

 Febrero
Table Tipo 1 - Cambiar de Frame y una tabla simple
No table Tipo 2
No table Tipo 3
2002

 Enero
Table Tipo 1 - Cambiar de Frame y una tabla

In [28]:
import os

In [53]:
all_files = os.listdir(r"B_RawData\sbs\B_RawData\cajas_municipales")

date = all_files[0].split("table_clean_")[1].split(".")[0]

all_dates = []
for namefile in all_files:
    date = namefile.split("table_clean_")[1].split(".")[0]
    all_dates.append( date )

df1 = pd.DataFrame( { "Date" : all_dates})

df2 = pd.concat( (df1, df1.Date.str.split("_", expand=True)), axis = 1).rename( columns = {  1 : "Month", 0 : "Year" })

years = df2.Year.unique()
for yr in years:
    
    n = df2[ df2.Year == yr ].shape[0]
    
    if n != 12 :
        print(yr)

2002
2022
